In [1]:
lines = sc.textFile('./data/uid_iid')

In [2]:
# target, etime, subset = "12-19-0", "12-18-23", {}
target, etime, subset = "12-18-0", "12-17-23", {}
# target, etime, subset = "12-17-0", "12-16-23", {}
# target, etime, subset = "12-16-0", "12-15-23", {}
# target, etime, subset = "12-15-0", "12-14-23", {}
# target, etime, subset = "12-14-0", "12-13-23", {}
# target, etime, subset = "12-13-0", "12-12-23", {}
# target, etime, subset = "12-12-0", "12-11-23", {}
# target, etime, subset = "12-11-0", "12-10-23", {}
# target, etime, subset = "12-10-0", "12-09-23", {}
# target, etime, subset = "12-09-0", "12-08-23", {}
# target, etime, subset = "12-08-0", "12-07-23", {}
# target, etime, subset = "12-07-0", "12-06-23", {}
# target, etime, subset = "12-06-0", "12-05-23", {}
# target, etime, subset = "12-05-0", "12-04-23", {}
# target, etime, subset = "12-04-0", "12-03-23", {}
# target, etime, subset = "12-03-0", "12-04-23", {}
# target, etime, subset = "12-02-0", "12-01-23", {}
# target, etime, subset = "12-01-0", "11-30-23", {}

In [3]:
import fileinput

In [4]:
for line in fileinput.input("./data/tianchi_fresh_comp_train_item.csv"):
    subset[line.split(",")[0]] = True

In [5]:
def extract1(line):
    import time
    (uid, iid, ict) = line.strip().split("\t")[0].split(" ")
    if subset.has_key(iid):
        items = filter(lambda x:x[0]>0, [(int(time.mktime(time.strptime('2014-'+etime,'%Y-%m-%d-%H'))-time.mktime(time.strptime('2014-'+i.split(",")[0],'%Y-%m-%d-%H')))/(24*3600)+1, int(i.split(",")[1])) for i in line.strip().split("\t")[1].split(" ")])
        return (iid,items)
    else:
        return ("","")

In [6]:
def extract2(items_list):
    items, items_buy, items_buy_3, f, inf = [], [], [], [0]*37, 100
    f[30] = len(items_list) # number of people who have interactions
    for i in items_list:
        if len(filter(lambda x:x[1]==4,i))>0:
            items_buy.append(i)
        if len(filter(lambda x:x[1]==4 and x[0]<=3,i))>0:
            items_buy_3.append(i)
        items.extend(i)
    f[31] = len(items_buy) # number of people who buy the item
    f[32] = len(items_buy_3) # number of people who buy the item in last three days
    f[33] = len(filter(lambda x:len(x)==1,items_list)) # number of people who have one interaction
    f[34] = len(filter(lambda x:len(x)==2,items_list)) # number of people who have two interactions
    f[35] = len(filter(lambda x:len(x)==3,items_list)) # number of people who have three interactions
    items = sorted(items, key=lambda x:x[0], reverse=True)
    f[0] = len(filter(lambda x:x[0]<=1 and x[1]==1, items)) # number of click on last day
    f[1] = len(filter(lambda x:x[0]<=1 and x[1]==2, items)) # number of collect on last day
    f[2] = len(filter(lambda x:x[0]<=1 and x[1]==3, items)) # number of cart on last day
    f[3] = len(filter(lambda x:x[0]<=1 and x[1]==4, items)) # number of buy on last day
    f[4] = len(filter(lambda x:x[0]==2 and x[1]==1, items)) # number of click on penultimate day
    f[5] = len(filter(lambda x:x[0]==2 and x[1]==2, items)) # number of collect on penultimate day
    f[6] = len(filter(lambda x:x[0]==2 and x[1]==3, items)) # number of cart on penultimate day
    f[7] = len(filter(lambda x:x[0]==2 and x[1]==4, items)) # number of buy on penultimate day
    f[8] = len(filter(lambda x:x[0]==3 and x[1]==1, items)) # number of click on third last day
    f[9] = len(filter(lambda x:x[0]==3 and x[1]==2, items)) # number of collect on third last day
    f[10] = len(filter(lambda x:x[0]==3 and x[1]==3, items)) # number of cart on third last day
    f[11] = len(filter(lambda x:x[0]==3 and x[1]==4, items)) # number of buy on third last day
    f[12] = len(filter(lambda x:x[0]<=7 and x[1]==1, items)) # number of click last week
    f[13] = len(filter(lambda x:x[0]<=7 and x[1]==2, items)) # number of collect last week
    f[14] = len(filter(lambda x:x[0]<=7 and x[1]==3, items)) # number of cart last week
    f[15] = len(filter(lambda x:x[0]<=7 and x[1]==4, items)) # number of buy last week
    f[16] = len(filter(lambda x:x[0]<=14 and x[1]==1, items)) # number of click last two weeks
    f[17] = len(filter(lambda x:x[0]<=14 and x[1]==2, items)) # number of collect last two weeks
    f[18] = len(filter(lambda x:x[0]<=14 and x[1]==3, items)) # number of cart last two weeks
    f[19] = len(filter(lambda x:x[0]<=14 and x[1]==4, items)) # number of buy last two weeks
    f[20] = min(1.0,round(1.0*(f[3]+f[7]+f[11])/(f[0]+f[4]+f[8]),4)) if (f[0]+f[4]+f[8]) else 0.0 # click conversion rate last three days
    f[21] = min(1.0,round(1.0*(f[3]+f[7]+f[11])/(f[1]+f[5]+f[9]),4)) if (f[1]+f[5]+f[9])!=0 else 0.0 # collect conversion rate last three days
    f[22] = min(1.0,round(1.0*(f[3]+f[7]+f[11])/(f[2]+f[6]+f[10]),4)) if f[2]!=0 else 0.0 # cart conversion rate last three days
    f[23] = min(1.0,round(1.0*f[7]/f[4],4)) if f[4]!=0 else 0.0 # click conversion rate last two weeks
    f[24] = min(1.0,round(1.0*f[7]/f[5],4)) if f[5]!=0 else 0.0 # collect conversion rate last two weeks
    f[25] = min(1.0,round(1.0*f[7]/f[6],4)) if f[6]!=0 else 0.0 # cart conversion rate last two weeks
    buy = filter(lambda x:x[1]==4, items)
    last = buy[-1][0] if len(buy)!=0 else inf
    f[26] = len(buy) # total number of buy 
    f[27] = len(filter(lambda x:x[0]==last and x[1]==1, items)) # number of click on the last day of buy
    f[28] = len(filter(lambda x:x[0]==last and x[1]==2, items)) # number of collect on the last day of buy
    f[29] = len(filter(lambda x:x[0]==last and x[1]==3, items)) # number of cart on the last day of buy
    f[36] = round(1.0*len(items)/f[30],4) if f[30]!=0 else 0.0 # average interations
    return "\t".join([str(i) for i in f])

In [7]:
counts = lines.map(lambda x : extract1(x))\
                .filter(lambda x : x[0]!="")\
                .groupByKey()\
                .map(lambda x : x[0]+"\t"+extract2(x[1]))

In [8]:
output = counts.saveAsTextFile("./feature/"+target+"/item/")

Traceback (most recent call last):
  File "/tmp/kernel-PySpark-2396b07d-4fb7-4e3f-86de-1f74e11cce00/pyspark_runner.py", line 205, in <module>
    output.reset()
AttributeError: 'NoneType' object has no attribute 'reset'